In [ ]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH
#DOWNLOAD_DATA = True

# To train set to true, to use pretrained model set to false
RETRAIN = False
LOCAL_TRAIN = True # Only for Local Linear Model
# To avoid out of memory error, adjust 
NUM_THREADS = 3

In [ ]:
import os.path,sys
sys.path.append(INIT_PATH)

import networks as nnets
import importlib
importlib.reload(nnets)

import pandas as pd, torch, torchvision,matplotlib, matplotlib.pyplot as plt, numpy as np
from typing import Tuple, List, Dict, Union

from time import sleep
import json
from IPython.display import display, HTML
torch.set_default_dtype(torch.float16)

In [ ]:
DATA_PATH = "." # Path to data folder
if (DOWNLOAD_DATA):
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


In [ ]:
trainer = nnets.ThreadedTrainer(
    num_threads= NUM_THREADS,
    DATA_PATH=DATA_PATH,
    SAVE_RESULT_PATH= INIT_PATH + "/files/results.json",
    SAVE_NETWORK_PATH=INIT_PATH + "/networks_saved/",
    SAVE_NON_ZERO_PATH = INIT_PATH + "/files/non_zero.json",
    saving_step= 20
)

### Optimization


You may use this script to test different hyperparameters
Load the network description in the file files/optim.json
The file is a json file with the following structure :
```json
{
    "optim":{ // Hyperparameters to optimize
        "model":{ // Hyperparameters to optimize for the model
            "S-LOCAL":{ // Model you want to optimize hyperparameters on 
                "hidden_size":[64,128,256,512], // List of values to test
                "dropout":[0.1,0.2,0.3,0.4,0.5] // List of values to test
            }
        }
    },
    "networks":{ // List of networks to test
        "S-LOCAL-64-0.1":{ // Name of the network
            //add the network default description here
        }
    },
    "results":{ // Results of the optimization
        "S-LOCAL-64-0.1":{ //Name of the network
            "best":{ 
                // Best hyperparameters found
            },
            "accuracy":0.9 // Accuracy of the best hyperparameters
        }
    }
}
```

In [ ]:

optim = json.load(open(INIT_PATH+"/files/optim.json"))

list_to_do = [key for key,item in optim["networks"].items() if
            key not in optim["results"] and
            not (not LOCAL_TRAIN and item["model"]["name"] in ["S-LOCAL","D-LOCAL"])]

for key in list_to_do:
    nnets.optimization_thread(optim,key, trainer)
    